# 人脸追踪 face follow play

### 导入头文件 Import head file

In [ ]:
import cv2 as cv
import threading
from time import sleep
import ipywidgets as widgets
from IPython.display import display
from face_follow import face_follow

### 初始化机械臂位置 Initialize DOFBOT position

In [ ]:
import Arm_Lib
Arm = Arm_Lib.Arm_Device()
joints_0 = [90, 150, 20, 20, 90, 30]
Arm.Arm_serial_servo_write6_array(joints_0, 1500)

### 创建实例,初始化参数 Create the instance and initialize the parameters

In [ ]:
follow = face_follow()
model = 'General'

### 创建控件 Creating widget

In [ ]:
button_layout = widgets.Layout(width='250px', height='50px', align_self='center')
output = widgets.Output()
# 退出控件 exit button
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
# 图像控件 Image widget
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
# 空间布局 spatial distribution
controls_box = widgets.VBox([imgbox, exit_button], layout=widgets.Layout(align_self='center'))
# ['auto', 'flex-start', 'flex-end', 'center', 'baseline', 'stretch', 'inherit', 'initial', 'unset']

### 模式切换 Switch mode

In [ ]:
def exit_button_Callback(value):
    global model
    model = 'Exit'
#     with output: print(model)
exit_button.on_click(exit_button_Callback)

### 主程序 Main process

In [ ]:
def camera():
    global model
    # 打开摄像头 Open camera
    capture = cv.VideoCapture(0)
    while capture.isOpened():
        try:
            _, img = capture.read()
            img = cv.resize(img, (640, 480))
            img = follow.follow_function(img)
            if model == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except KeyboardInterrupt:capture.release()

### 启动  Start

In [ ]:
display(controls_box,output)
threading.Thread(target=camera, ).start()